# Inferencja w generatorze RNN przy użyciu GPU lub CPU

In [ ]:
from pathlib import Path
dataset_path = Path.home() / 'data/rnn_generator'; dataset_path
tmp_path = dataset_path / 'tmp/'

In [ ]:
import torch

USE_GPU = torch.cuda.is_available(); 
# USE_GPU = False; 

print(f'USE_GPU={USE_GPU}')

def to_gpu(x, *args, **kwargs):
    return x.cuda(*args, **kwargs) if USE_GPU else x

In [ ]:
ALLCHARS, MODEL = ['all_characters', 'model']

fn_witkacy = {ALLCHARS: 'all_characters.p', MODEL: 'model.h100.l1.e2000.cpu.dat'}
# fn_pan_tadeusz = {ALLCHARS: 'all_characters.pan_tadeusz.p', MODEL: 'pan_tadeusz.h100.l1.e2000.cpu.torch'}
# fn_pan_tadeusz = {ALLCHARS: 'all_characters.pan_tadeusz.p', MODEL: 'pan_tadeusz.h100.l1.e101.gpu.torch'}
fn_pan_tadeusz = {ALLCHARS: 'all_characters.pan_tadeusz.p', MODEL: 'pan_tadeusz.h400.l3.3000.gpu.torch'}


fn_dict = fn_pan_tadeusz

In [ ]:
tmp_path

In [ ]:
ls -lah $tmp_path

## Ładowanie listy all_characters

In [ ]:
import pickle

all_characters = pickle.load( open( tmp_path / fn_dict[ALLCHARS], 'rb' ) ); print(all_characters[:10])
n_characters = len(all_characters); print(n_characters)

## Kod

In [ ]:
import torch
import torch.nn as nn
from torch.autograd import Variable

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, n_layers=1):
        super(RNN, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        
        self.encoder = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers)
        self.decoder = nn.Linear(hidden_size, output_size)
    
    def forward(self, input, hidden):
        input = self.encoder(input.view(1, -1))
        output, hidden = self.gru(input.view(1, 1, -1), hidden)
        output = self.decoder(output.view(1, -1))
        return output, hidden

    def init_hidden(self):
        return Variable(to_gpu(torch.zeros(self.n_layers, 1, self.hidden_size)))

In [ ]:
# Turn string into list of longs
def char_tensor(string):
    tensor = torch.zeros(len(string)).long()
    for c in range(len(string)):
        tensor[c] = all_characters.index(string[c])
    return Variable(to_gpu(tensor))

print(char_tensor('ala ma kota'))

In [ ]:
def evaluate(prime_str='A', predict_len=100, temperature=0.8):
    hidden = decoder.init_hidden()
    prime_input = char_tensor(prime_str)
    predicted = prime_str

    # Use priming string to "build up" hidden state
    for p in range(len(prime_str) - 1):
        _, hidden = decoder(prime_input[p], hidden)
    inp = prime_input[-1]
    
    for p in range(predict_len):
        output, hidden = decoder(inp, hidden)
        
        # Sample from the network as a multinomial distribution
        output_dist = output.data.view(-1).div(temperature).exp()
        top_i = torch.multinomial(output_dist, 1)[0]
        
        # Add predicted character to string and use as next input
        predicted_char = all_characters[top_i]
        predicted += predicted_char
        inp = char_tensor(predicted_char)

    return predicted

In [ ]:
n_epochs = 300 # 3000
print_every = 100
plot_every = 1
hidden_size = 100
n_layers = 1
lr = 0.005

decoder = RNN(n_characters, hidden_size, n_characters, n_layers)
print(decoder, flush=True)

## Wczytywanie modelu

In [ ]:
model_path_cpu = tmp_path / fn_dict[MODEL]
decoder = torch.load(model_path_cpu)
print(decoder, flush=True)

In [ ]:
decoder.gru.flatten_parameters()

## Generowanie tekstu

In [ ]:
# priming strings
# pan_tadeusz: kon, bę, Tad, Tadeusz

In [ ]:
print(evaluate('Tadeusz', 1000, temperature=0.8))

In [ ]:
print(evaluate('Tad', 1000, temperature=0.5))

In [ ]:
print(evaluate('bę', 200, temperature=1.4))